In [1]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import lightgbm as lgb
# Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestRegressor,  
                               ExtraTreesRegressor)
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [2]:
%%time
df_train = pd.read_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/ultimate2/train_test/train_features.csv")
df_test = pd.read_csv("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/ultimate2/train_test/test_features.csv")

Wall time: 14.5 s


In [9]:
df_train.shape

(201917, 86)

In [3]:
def feature_concat(train_data1,filepath):
    for file in filepath:
        data_features = pd.read_csv(file)
        data_features = data_features.drop_duplicates()
        train_data1=pd.merge(train_data1,data_features,how='left',on='card_id')
#         print(file)
#         print(train_data1.shape)
    return(train_data1)

In [4]:
# Put in our parameters for said classifiers
lgb_param1 = {
    'max_leaves':100,
             'min_data_in_leaf': 177, 
             'max_depth': 60,
            'feature_fraction':0.8,
             'objective':'regression',
             'learning_rate': 0.01,
             "boosting": "gbdt",
             "bagging_seed": 11,
             "metric": 'rmse',
             "lambda_l1": 0.1,
             "verbosity": -1,
             "nthread": 4,
             "random_state": 4590}
### Lightgbm
def run_lgb1(df_train,df_test,df_train_columns,lgb_params):
    param =  lgb_params
    val_metrics=[]
    oof_test_skf = np.empty((5, len(df_test)))
    folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)
    oof_train = np.zeros(len(df_train))
    oof_test = np.zeros(len(df_test))
    feature_importance_df = pd.DataFrame()
    print(df_train.shape)
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train,df_train['outliers'].values)):
        print("fold {}".format(fold_))
        trn_data = lgb.Dataset(df_train.iloc[trn_idx][df_train_columns], label=target.iloc[trn_idx])#, categorical_feature=categorical_feats)
        val_data = lgb.Dataset(df_train.iloc[val_idx][df_train_columns], label=target.iloc[val_idx])#, categorical_feature=categorical_feats)
#         print(trn_data.shape,val_data.shape)
        num_round = 3000
        clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
        oof_train[val_idx] = clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
#
        fold_importance_df = pd.DataFrame()
        fold_importance_df["Feature"] = df_train_columns
        fold_importance_df["importance"] = clf.feature_importance()
        
        fold_importance_df["fold"] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

        test_predictions = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)
        oof_test_skf[fold_, :] = test_predictions
        
        predictions= clf.predict(df_train.iloc[val_idx][df_train_columns], num_iteration=clf.best_iteration)
        val_metrics.append(np.sqrt(mean_squared_error(predictions, target.iloc[val_idx])))
    oof_test[:] = oof_test_skf.mean(axis=0)
    return feature_importance_df,oof_train.reshape(-1,1),oof_test.reshape(-1,1),val_metrics

In [ ]:
# import glob
# folder ="./ultimate2/features_80/*"
# folder_list=glob.glob(folder)
# valid_results={}
# file_list=[]
# for folder in folder_list:
#     file_list1=glob.glob(folder+"/*.csv")
#     file_list.extend(file_list1)
# train_features=feature_concat(df_train,file_list)
# test_features=feature_concat(df_test,file_list)
# numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
# train_features.shape


In [12]:
import glob
file_list = glob.glob("C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/features_90/features_90/*.csv")
train_features=feature_concat(df_train,file_list)
test_features=feature_concat(df_test,file_list)
print(train_features.shape)
target = df_train['target']
train_features=train_features.fillna(0)
test_features=test_features.fillna(0)

C:\Users\HP\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2802: DtypeWarning: Columns (1,9,17,30,41,50,62,71,79,89,99,108,120,130,138,151) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


(201917, 605)


In [13]:
test_featurest_features.shape

(123623, 603)

## Model Run

In [17]:
all_results=[]
for i in range(0,4):
    print("run",i)
    if i==0:
        numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
        train_features = train_features.select_dtypes(include=numerics)
        train_features.columns = [x.replace("<lambda>","range") for x in train_features.columns]
        test_features.columns = [x.replace("<lambda>","range") for x in test_features.columns]
        df_train_columns = [c for c in train_features.columns if c not in ['card_id', 'first_active_month','target','outliers','class_outcome']]
    if i==1:
        df_train_columns = [c for c in train_features.columns if c not in ['card_id', 'first_active_month','target','outliers']]
    if i==2:
        df_train_columns = imp_features1.copy()
    if i==3:
        df_train_columns.extend(['class_outcome'])
    if i==0:
        imp_features1 = imp_features.groupby('Feature').agg({'importance':sum}).reset_index().sort_values(by = 'importance', ascending = False).head(250)['Feature'].tolist()
    imp_features,lgb_oof_train, lgb_oof_test,val_metrics_gb=run_lgb1(train_features,test_features,df_train_columns, lgb_param1)
    print(val_metrics_gb)
    all_results.append([imp_features,lgb_oof_train, lgb_oof_test,val_metrics_gb])
import pickle
pickle.dump( all_results, open( "C:/Kaggle competetion datasets/Elo Merchent Category Recommendation/Elo Feb 18th/ultimate2/pickle_results/results_85.pickle", "wb" ) )



run 0
(201917, 581)
fold 0
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3.61417	valid_1's rmse: 3.70085
[200]	training's rmse: 3.50625	valid_1's rmse: 3.66777
[300]	training's rmse: 3.42952	valid_1's rmse: 3.65694
[400]	training's rmse: 3.36832	valid_1's rmse: 3.65293
[500]	training's rmse: 3.31323	valid_1's rmse: 3.65217
[600]	training's rmse: 3.2625	valid_1's rmse: 3.65163
[700]	training's rmse: 3.21591	valid_1's rmse: 3.65216
Early stopping, best iteration is:
[608]	training's rmse: 3.25832	valid_1's rmse: 3.65162
fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3.61792	valid_1's rmse: 3.69494
[200]	training's rmse: 3.51062	valid_1's rmse: 3.66574
[300]	training's rmse: 3.43034	valid_1's rmse: 3.65533
[400]	training's rmse: 3.36763	valid_1's rmse: 3.65171
[500]	training's rmse: 3.31198	valid_1's rmse: 3.64979
[600]	training's rmse: 3.26184	valid_1's rmse: 3.65008
Early stopping, best iteration is:
[

fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3.61999	valid_1's rmse: 3.6997
[200]	training's rmse: 3.50877	valid_1's rmse: 3.67186
[300]	training's rmse: 3.42909	valid_1's rmse: 3.66332
[400]	training's rmse: 3.36293	valid_1's rmse: 3.65993
[500]	training's rmse: 3.30319	valid_1's rmse: 3.65915
[600]	training's rmse: 3.24749	valid_1's rmse: 3.65868
[700]	training's rmse: 3.19659	valid_1's rmse: 3.65954
Early stopping, best iteration is:
[606]	training's rmse: 3.2445	valid_1's rmse: 3.65843
fold 2
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3.61674	valid_1's rmse: 3.70828
[200]	training's rmse: 3.50881	valid_1's rmse: 3.67625
[300]	training's rmse: 3.42955	valid_1's rmse: 3.666
[400]	training's rmse: 3.36419	valid_1's rmse: 3.65976
[500]	training's rmse: 3.30569	valid_1's rmse: 3.65732
[600]	training's rmse: 3.25106	valid_1's rmse: 3.65616
[700]	training's rmse: 3.19974	valid_1's rmse: 3.65613
Earl

In [18]:
# for i in range(0,4):
#     print("run",i)
#     if i==0:
#         numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#         train_features = train_features.select_dtypes(include=numerics)
#         train_features.columns = [x.replace("<lambda>","range") for x in train_features.columns]
#         test_features.columns = [x.replace("<lambda>","range") for x in test_features.columns]
#         df_train_columns = [c for c in train_features.columns if c not in ['card_id', 'first_active_month','target','outliers','class_outcome']]
#     if i==1:
#         df_train_columns = [c for c in train_features.columns if c not in ['card_id', 'first_active_month','target','outliers']]
#     if i==2:
#         df_train_columns = imp_features1.copy()
#     if i==3:
#         df_train_columns.extend(['class_outcome'])
#     print(len(df_train_columns))